# TME 8 : Mini-projet Detection de motifs


## Localisation de pattern (motifs) dans le genome

Nous allons implémenter plusieurs algorithmes afin de localiser un motif d'intérêt dans un génome.

### A -  Boyer Moore Algorithm 
1\. Nous allons réutiliser les fonctions du TME6 pour générer une séquence artificiel de taille ``n``, et implanter un motif de taille ``k`` dans une position aléatoire de la séquence. Vous pouvez aussi faire varier ``v`` positions pour permettre certaines mutations


In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

n = 100 #longuer des sequence
k = 8 #longuer du motif
v = 0 #nombre de variations

In [2]:
#generate and implant variavel motifs

def generateRandomSequence ( n, maj ):
    seq = ""
    for i in range ( n ):  
        seq += nuc [ random.randint ( 0, 3 ) ]
    if not ( maj ):
        seq = seq.lower()
    return seq


def generateRandomSequences(n, t):
    sequences = []
    for i in range ( t ):
        sequences.append ( generateRandomSequence ( n, True ) )
    return sequences

def replaceSeq ( sequence, v ):
    sequence = list ( sequence )
    for i in range ( v ):
        pos = random.randint ( 0, len ( sequence ) - 1 )
        nucleo = sequence [ pos ]
        while sequence [ pos ] == nucleo:
            sequence [ pos ] = random.choice ( nuc ) 
    return "".join ( sequence )

def implantMotifVar(k, v, t, n):
    motif = generateRandomSequence ( k, True )
    sequences = generateRandomSequences ( n, t )
    cpt = 0
    for seq in sequences:
        pos = random.randint ( 0, len ( seq ) - k )
        sub = replaceSeq ( motif, v )
        seq = seq [ 0 : pos ] + sub + seq [ len ( sub ) + pos : ]
        sequences [ cpt ] = seq
        cpt += 1
    return ( sequences, motif )

sequence, motif = implantMotifVar(k, v, 1, n)
sequence = sequence [0]
print ( "sequence =", sequence )
print ( "motif =", motif )

sequence = CGAGGTCTCGATTACCAGACCGCGCTGAGCTCGAGACTTGCTCGCGGCCGCTGCAGTGGACCCGGTGCAATAGTGACGTCCTTACGTTAGCTATAATAGT
motif = GCTATAAT


2\. Boyer Moore est une combinaison de deux approches : bad character et good Suffix. Nous allons commencer par la première approche. Définissez une fonction ``badCharacter`` pour construire la badTable d'un motif. Vous pouvez l'implémenter comme dictionnaire, voir un exemple ci-dessous.

<img src="boyer-moore-badTable.png">

In [3]:
def badCharacter(motif):
    '''
    The preprocessing function for Boyer Moore's bad character heuristic
    '''
    badChar = {}
    taille = len ( motif )
    for i in range ( len ( motif ) ):
        badChar [ motif [ i ] ] = max ( 1, taille - i - 1 )  
    return badChar

badCharacter("TATGTG") #{'T': 1, 'A': 4, 'G': 1}

{'T': 1, 'A': 4, 'G': 1}

3\. Développer la version de l'algorithme Boyer Moore qui utilise "bad character" la règle de caractère incorrect.

In [4]:
import sys
def search(genome, motif, verbose = True):
    '''
    A pattern searching function that uses Bad Character Heuristic of Boyer Moore Algorithm
    '''
    return search_aux ( genome, motif, badCharacter ( motif ), 0, 0, verbose )
    

def search_aux ( genome, motif, badTable, indice, comp_evitees, verbose ):
    if len ( motif ) > len ( genome ):
        return -1
    for i in range ( len ( motif ) - 1, -1, -1 ):
        if genome [ i ] not in badTable:
            return search_aux ( genome [ 1 :: ], motif, badTable, indice + 1, comp_evitees, verbose )
        if genome [ i ] != motif [ i ]:
            decalage = badTable [ motif [ i ] ]
            return search_aux ( genome [ decalage : ], motif, badTable, indice + decalage, comp_evitees + decalage, verbose )
    if verbose:
        print ( "Comparaisons évitées :", comp_evitees )
    return indice
            
#search("ABCABB", "ACB") #-1
                   
#search("ABCABB", "CA") #2

4\. Testez la version de l'exercice 3 sur les données artificielles généré dans 1, combien de comparaison inutile avez-vous évité avec cette version? 

In [5]:
search ( sequence, motif ) == sequence.find ( motif )

Comparaisons évitées : 89


True

5\. Nous allons implémenter la règle de good suffixe. Pour cela, nous allons pré-processer le motif afin de trouver tous les suffixe de taille k que sont aussi de préfixe. Faire la fonction goodSuffix qui étant donnée un motif renvoie un dictionnaire ou les clés sont de suffixes et les valeurs sont la distance au préfixe le plus proche, voir un exemple ci-dessous. d2 est la distance entre le suffixe de taille k et le son occurence la plus proche.

<img src="suffix.png">

In [6]:
def goodSuffix(motif):
    
    gs = {}
    
    for k in range ( len ( motif ) - 1, 0, -1 ):
        cle = motif [ -k :: ]
        gs [ cle ] = goodSuffixAux ( motif, cle )
        
   
    return gs

def goodSuffixAux ( motif, cle ):
    motif = motif [ :: -1 ] 
    motif2 = motif [ len ( cle ) :: ]
    cle2 = cle [ :: -1 ]
    while True:
        indice = motif2.find ( cle2 )
        if indice != -1:
            return len ( cle ) + indice
        cle2 = cle2 [ 0 : len ( cle2 ) - 1 ]
        if cle2 == "":
            cle2 = cle [ 1:: ]
            cle = cle [ 1 :: ]
            motif2 = motif [ len ( cle ) :: ]

    

gs = goodSuffix("ABCBAB"); print (gs)  #{'B': 2, 'AB': 4, 'BAB': 4, 'CBAB': 4, 'BCBAB': 4}
gs = goodSuffix("BAOBAB"); print (gs)  #{'B': 2, 'AB': 2, 'BAB': 5, 'OBAB': 5, 'AOBAB': 5}
print (gs)



{'BCBAB': 4, 'CBAB': 4, 'BAB': 4, 'AB': 4, 'B': 2}
{'AOBAB': 5, 'OBAB': 5, 'BAB': 5, 'AB': 2, 'B': 2}
{'AOBAB': 5, 'OBAB': 5, 'BAB': 5, 'AB': 2, 'B': 2}


6\. Développer la version de l'algorithme Boyer Moore qui utilise les deux règles "bad character" et  "good suffixe". Avez-vous amélioré votre algorithme? 

In [7]:
import sys
def search(genome, motif, verbose = True ):
    '''
    A pattern searching function that uses Bad Character
    Heuristic of Boyer Moore Algorithm
    '''
    badChar = badCharacter(motif) 
    gs = goodSuffix(motif)
    return search_aux ( genome, motif, badChar, gs, 0, 0, verbose )

def search_aux ( genome, motif, badTable, gs, indice, comp_evitees, verbose ):
    if len ( motif ) > len ( genome ):
        return -1
    for i in range ( len ( motif ) - 1, -1, -1 ):
        if genome [ i ] not in badTable:
            return search_aux ( genome [ 1 :: ], motif, badTable, gs, indice + 1, comp_evitees, verbose )
        if genome [ i ] != motif [ i ]:
            decalage = badTable [ motif [ i ] ]
            for j in range ( min ( len ( motif ), len ( genome ) ) - 1, 0, -1 ):
                seq = genome [ i : j ]
                if seq in gs:
                    decalage = max ( decalage, gs [ seq ] )
            return search_aux ( genome [ decalage : ], motif, badTable, gs, indice + decalage, comp_evitees + decalage, verbose )
    if verbose:
        print ( "Comparaisons évitées :", comp_evitees )
    return indice
    
               
search("ACAAABABABC", "ABABAB")

       

Comparaisons évitées : 4


4

L'algorithme évite un plus grand nombre de comparaisons inutiles avec cette nouvelle heuristique.

7\. Tester l'algorithme **Boyer Moore Algorithm** sur vos données de chipSeq. Vous pouvez chercher de motifs trouvés par les algorithms développé précédemment

In [8]:
def readFasta(genome):
    sequence = []
    file = open(genome, "r")
    sequences = []
    seq = ""
    for s in file:
        if s[0] != ">":
            seq += s.strip().upper()
        else:
            sequences.append(seq)
            seq = ""
    return sequences[1:]
genome = "Sequence_by_Peaks_6.fasta"
sequences = readFasta(genome)
#print(sequences)

In [9]:
motif = "CCAAT"

def afficher_nb_motifs_trouvés ( sequences, fonction_recherche, verbose = False ) :
    cpt = 1
    occurrences = 0
    for sequence in sequences:
        indice = fonction_recherche ( sequence, motif, verbose )
        if indice == -1:
            if verbose :
                print ( "Motif introuvable dans la séquence", cpt )
        else:
            if verbose :
                print ( "Motif trouvé à la position", indice, "de la séquence", cpt )
            occurrences += 1
        cpt += 1
    print ( "Le motif", motif, "a été trouvé dans", occurrences, "séquences différentes." )
    
afficher_nb_motifs_trouvés ( sequences, search )

Le motif CCAAT a été trouvé dans 23 séquences différentes.


### B -  Index of fixed length words
8\. Faire une fonction pour indexer les positions d'occurrences de tous les mots de taille k dans un texte, la fonction doit renvoyer un dictionnaire ou les clés sont les mots et les valeurs les positions dans le texte.

In [10]:
def indexTable(k, texte):
    indexes  = {};
    for i in range ( len ( texte ) - k ):
        mot = texte [ i : i + k ]
        if mot not in indexes:
            indexes [ mot ] = [ i ]
        else:
            indexes [ mot ].append ( i )
    
    return indexes

#test 
indices = indexTable ( 3, "ACGGCTAGTTAGAA" )
#print ( indices )


9\. Faire une fonction pour découper un motif en sous-parties de taille k

In [11]:
def seeds(k, motif):
    """On choisit de découper les motifs en commençant par la fin"""
    graines = []
    for i in range ( len ( motif ), 0, -k ):
        decoupage = ( motif [ i - k : i ] )
        if len ( decoupage ) > 0:
            graines.append ( decoupage )
    reste = len ( motif ) % k
    if reste != 0:
        graines.append ( "." * ( k - reste ) + motif [ 0 : reste ] )
    return graines
    
#test
seeds ( 3, "GTTAG" )

['TAG', '.GT']

10\. Faire une fonction pour chercher un motif dans un texte utilisant la table des indexes de la question 7

In [12]:
import re

def comparer_mots ( tableDesIndices, regexp, indice_a_trouver ):
    for mot, indices in tableDesIndices.items():
        if re.search ( regexp, mot ) == None:
            continue
        for indice in indices:
            if indice == indice_a_trouver:
                lettres_manquantes = 0
                for lettre in regexp:                    
                    if lettre == ".":                       
                        lettres_manquantes += 1
                return indice + lettres_manquantes
    return -1 

def searchIndexTable(indexTable, k, motif, verbose = True):
    graines = seeds ( k, motif ) 
    indice = -1
    fin_motif = graines [ 0 ]
    comp_evitees = 0
    if fin_motif not in indexTable:
        return -1
    for candidat in indexTable [ fin_motif ]:
        indice = candidat
        comp_evitees = indice - comp_evitees
        for mot in graines [ 1 :: ]:
            indice = comparer_mots ( indexTable, mot, indice - k )
        if indice != -1:
            break
    if verbose:
        print ( "Comparaisons évitées :", comp_evitees )
    return indice

def recherche_par_indices ( sequence, motif, verbose = False ):
    indices = indexTable ( k, sequence )
    return searchIndexTable ( indices, k, motif, verbose )

#test 

texte = "ACGGCTAGTTAGAA"
motif = "GTTAG"
k = 3
pos = recherche_par_indices ( texte, motif )
assert ( texte [ pos : pos + len ( motif ) ] == motif )

11\. Tester l'algorithme **Index of fixed length** words sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [13]:
sequences = readFasta ( genome )
motif = "CCAAT"
afficher_nb_motifs_trouvés ( sequences, recherche_par_indices, False ) 

Le motif CCAAT a été trouvé dans 23 séquences différentes.


### C -  Matrice de fréquences
12\.Nous allons implémenter la recherche de motif par matrice de fréquence. Nous allons utiliser les matrices déjà fabriqué que vous pouvez telecharcher du site http://jaspar.genereg.net/
Une fois que vous avez chargé votre matrice de fréquence, vous devez la transformer en matrice de probabilité

In [14]:
import numpy as np

nuc = ('A', 'C', 'G', 'T')


matrice = np.array([[462.00,24.00,0.00,1000.00,0.00,7.00,90.00,600.00],
[86.00,16.00,22.00,0.00,1000.00,0.00,198.00,138.00],
[387.00,0.00,803.00,0.00,0.00,993.00,0.00,162.00],
[65.00,960.00,175.00,0.00,0.00,0.00,712.00,100.00]])

#Computing wi(a)
def pwm(M):
    res = np.zeros ( ( 4, len ( M [ 0 ] ) ) )
    M = profile ( M )
    for i in range ( len ( M ) ):
        for j in range ( len ( M [ i ] ) ):
            col = M [ 0 : len ( M ), i ]
            somme = col.sum()
            res [ i ] [ j ] = M [ i ] [ j ] / somme
    return res

def profile ( M ):
    for i in range ( len ( M ) ):
        for j in range ( len ( M [ i ] ) ):
            M [ i ] [ j ] += 1
    return M

k = 8
PWM = pwm(matrice); print (PWM)


[[4.61155378e-01 2.49003984e-02 9.96015936e-04 9.97011952e-01
  9.96015936e-04 7.96812749e-03 9.06374502e-02 5.98605578e-01]
 [8.66533865e-02 1.69322709e-02 2.29083665e-02 9.96015936e-04
  9.97011952e-01 9.96015936e-04 1.98207171e-01 1.38446215e-01]
 [3.86454183e-01 9.96015936e-04 8.00796813e-01 9.96015936e-04
  9.96015936e-04 9.90039841e-01 9.96015936e-04 1.62350598e-01]
 [6.57370518e-02 9.57171315e-01 1.75298805e-01 9.96015936e-04
  9.96015936e-04 9.96015936e-04 7.10159363e-01 1.00597610e-01]]


12\. Déterminer les paramètres f(0)(b) du modèle nul, où 
\begin{equation}
f^{(0)}(b) = \frac 1L \sum_{i=0}^{L-1} \omega_i(b)\ ,
\end{equation}

In [36]:

def f0_calcule(PWM, L):
    res = [ 0 ] * 4
    for i in range ( L ):
        for j in range ( len ( PWM ) ):
            res [ j ] += PWM [ j ] [ i ]
    return [ r / L for r in res ]    

f_0 = f0_calcule(PWM, k)
print (f_0)


[0.2727838645418327, 0.18276892430278885, 0.29295318725099606, 0.25149402390438247]


13\.Faites une fonction pour calculer log-rapport de vraisemblancee, d'une sequence de taille k.
\begin{equation}
\label{eq:ll}
\ell(b_0,...,b_{k-1}) = \log_2 \frac {P(b_0,...,b_{k-1} | \omega )
}{P^{(0)}(b_0,...,b_{k-1})}
= \sum_{i=0}^{k-1} \log_2 \frac {\omega_i(b_i)}{f^{(0)}(b_i)}\ .
\end{equation}

In [37]:
from math import log

def loglikehood(seq, PWM, f_0):
    res = 0
    for i in range ( len ( seq ) ):
        if seq [ i ] == 'A':
            j = 0
        elif seq [ i ] == 'C':
            j = 1
        elif seq [ i ] == 'G':
            j = 2
        else:
            j = 3
        res += log ( PWM [ j ] [ i ] / f_0 [ j ] )
    return res

loglikehood("ATGACGTA", PWM, f_0)

8.901561599704612

14\.Tester l'algorithme sur vos données de chipSeq. Vous devez parcourir les séquences et extraire les motifs ayant  log-rapport de vraisemblance positive 

Nous allons créer un profil pour le motif CCAAT à partir de la fonction inexact_match du TME7

In [38]:
#Création des k-mers valides

from itertools import product

k = 5

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for motif in motifs:
        if motif.count ( 'A' ) >= minrep:
            continue
        if motif.count ( 'T' ) >= minrep:
            continue
        if motif.count ( 'C' ) >= minrep:
            continue
        if motif.count ( 'G' ) >= minrep:
            continue
        validMotif.append ( motif )
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    for motif in motifs:
        rep = False
        for i in range ( len ( motif ) - 3 ):
            if motif [ i : i + 2 ] == motif [ i + 2 : i + 4 ]:
                rep = True
                break
        if not ( rep ):
            validMotif.append ( motif )
    return validMotif

def generateKmers(alphabet, k):
    validKmers = []
    res = []
    allkmers = product ( alphabet, repeat=k )
    for kmer in allkmers :
        validKmers.append ( kmer )
    validKmers = removeLowComplexe ( validKmers, (int) (k/2) + 1 )
    validKmers = removeLowComplexePair ( validKmers )
    for kmer in validKmers:
        res.append ( "".join ( kmer ) )
    return res

kmersValid = generateKmers(nuc, k)
#print ( kmersValid [0:100] )
#print ( len ( kmersValid ) )

In [39]:
def hamdist(v, x):
    res = 0
    for i in range ( len ( v ) ):
        if v [ i ] != x [ i ]:
            res += 1
    return res

def inexact_match(kmersV, sequences, stree, v):
    sequence = "".join ( sequences )
    nb_graines = v + 1
    taille_graine = ( int ) ( k / nb_graines )
    j = 0
    candidats = dict()
    for kmerV in kmersV:
        for i in range ( nb_graines ):
            depart = i * taille_graine
            graine = kmerV [ depart : depart + taille_graine ]
            indices_candidats = stree.find_all ( graine )
            for indice in indices_candidats:
                seq_candidate = sequence [ indice - depart : indice + k - depart ]
                if len ( seq_candidate ) == k and hamdist ( kmerV, seq_candidate ) <= v:
                    if kmerV in candidats:
                        variations = candidats [ kmerV ] [ 0 ] + [ seq_candidate ] 
                        occurences = candidats [ kmerV ] [ 1 ] + 1
                        candidats [ kmerV ] = ( variations, occurences )
                    else:
                        candidats [ kmerV ] = ( [ seq_candidate ] , 1 )
    return dict ( sorted ( candidats.items(), key=lambda motif : -motif [ 1 ] [ 1 ] ) )  

In [40]:
#Récupération des occurrences des motifs avec 2 variations et construction de l'arbre

from suffix_trees import STree

def construct_tree(sequences):
    st = STree.STree ( "".join ( sequences ) )
    return st

v = 2
st = construct_tree(sequences)
motif_occur_sorted = inexact_match(kmersValid, sequences, st, v)

In [41]:
#On récupère les occurrences du motif CCAAT
motifs = motif_occur_sorted["CCAAT"][0]
print ( motifs )

['CCTGT', 'CAAAT', 'CCAAC', 'CCCCT', 'CGACT', 'CCATT', 'CCAAT', 'CCGAT', 'CCTGT', 'CTTAT', 'CCCAT', 'CCATT', 'CCTTT', 'CAAAA', 'CCATC', 'CTAAT', 'CCGTT', 'CCGGT', 'CCTCT', 'CTAAA', 'CCACC', 'CCATC', 'CCTCT', 'CCCCT', 'CGAAA', 'CCAAA', 'CAAAA', 'CCCAC', 'CCACA', 'CGAAA', 'CAAAT', 'CAAAA', 'CTAAC', 'CAGAT', 'CATAT', 'CCGAG', 'CGAAG', 'CTAAC', 'CCGGT', 'CCCAC', 'CCACG', 'CGAAT', 'CAAAT', 'CCACT', 'CCATT', 'CTAAT', 'CCAGG', 'CCAGG', 'CAAAA', 'CCGCT', 'CTCAT', 'CCCAT', 'CCATT', 'CCGGT', 'CCAGC', 'CGAGT', 'CTCAT', 'CCTAT', 'CCGAA', 'CGAAG', 'CCTTT', 'CCGGT', 'CGTAT', 'CCGAC', 'CGACT', 'CAAAG', 'CCAAT', 'CAATT', 'CTAAT', 'CAAAG', 'CCTTT', 'CAATT', 'CCAAT', 'CAATT', 'CCAGT', 'CCAAA', 'CAAAA', 'CCAAA', 'CAAAG', 'CAAAT', 'CCATT', 'CTAAA', 'CAAAA', 'CGAGT', 'CTCAT', 'CAAAA', 'CTTAT', 'CTATT', 'CTAAA', 'CAATT', 'CCGAA', 'CGAAG', 'CTAGT', 'CCTAA', 'CTAAG', 'CTTAT', 'CGTAT', 'CCAAT', 'CAATT', 'CCAGT', 'CTATT', 'CTAAC', 'CCAGT', 'CTACT', 'CACAT', 'CATAT', 'CCCTT', 'CAAAA', 'CTACT', 'CAAAA', 'CCCAA', 

In [43]:
#On créé à présent le profil du motif

k = 5

def profile2(motifs, k, t):
    q = len(nuc)
    PWM = np.ones((q, k))#pseudo count
    
    for i in range ( k ):
        for motif in motifs:
            if motif [ i ].upper() == 'A':
                PWM [ 0 ] [ i ] += 1
            if motif [ i ].upper() == 'C':
                PWM [ 1 ] [ i ] += 1
            if motif [ i ].upper() == 'G':
                PWM [ 2 ] [ i ] += 1
            if motif [ i ].upper() == 'T':
                PWM [ 3 ] [ i ] += 1
                
    return PWM

p = profile2 ( motifs, k, len ( motifs ) )
print ( p )

[[ 878. 1038. 4420. 3433. 1128.]
 [3182. 3421.  374.  545.  558.]
 [ 529.  341.  198.  564.  633.]
 [ 839.  628.  436.  886. 3109.]]


In [54]:
def extract_indexes ( sequence, PWM, f_0, k ):
    res = []
    trouve = 0
    for i in range ( len ( sequence ) - k ):
        if trouve != 0:
            trouve -= 1
            continue
        seq = sequence [ i : i + k ]
        log=loglikehood(seq,PWM,f_0)
        if log > 0:
            res.append ( ( i, seq,log) )
            trouve = k
    return res

def extract_indexes_sequences ( sequences, PWM, f_0, k ):
    res = dict()
    cpt = 1
    for sequence in sequences:
        cle = "Sequence n° " + str ( cpt )
        res [ cle ] = extract_indexes ( sequence, PWM, f_0, k)
        cpt += 1
    return res

In [58]:
PWM = pwm ( p )
f_0 = f0_calcule ( PWM, k )
positions_motif = extract_indexes_sequences ( sequences, PWM, f_0, k )
print ( positions_motif )


{'Sequence n° 1': [(3, 'CCCTG', 0.02196657927787865), (17, 'GCTGG', 0.46123321195109623), (32, 'GCAAG', 2.3862546833997564), (38, 'GTATT', 0.9142429654806326), (45, 'TGATG', 0.135036325945108), (57, 'ATAAT', 0.5954580357600141), (69, 'CCTTT', 1.1201297581716898), (87, 'CCTCT', 0.3277390548128407), (96, 'GTATG', 0.2788507514036933), (113, 'CCTCT', 0.3277390548128407), (124, 'GGAGT', 1.7594015265334555), (133, 'CGAAA', 0.9003024827923322), (141, 'TTATT', 0.42429850925992996), (149, 'ACAGA', 0.10951796444344897), (157, 'TCATA', 0.17591962689217755), (165, 'TCATA', 0.17591962689217755), (176, 'CCACG', 1.3798071232357483), (200, 'ACAAT', 1.9662722485165585), (208, 'GCCAG', 0.23602343608303727), (221, 'CCGCT', 0.5073500977627908), (230, 'CCGGT', 1.7987693314123456), (237, 'AGAAG', 0.30619585244519226), (246, 'CCTAT', 1.8567194774112088), (273, 'CCGGT', 1.7987693314123456), (280, 'CCGAC', 0.019011102639911126), (294, 'ACAGG', 1.0933188454908063), (305, 'CCAAT', 3.5441797599110556), (314, 'TTA

In [123]:
import operator
#Retourne toutes les séquences avec leur log respectif
def get_max_log(sequences,PWM,f_0,k):
    positions_motifs=extract_indexes_sequences(sequences,PWM,f_0,k)
    print("len : ",len(positions_motifs))
    log_max=dict()
    tmp=[]
    cpt=0
    for sequence,L in positions_motifs.items():
        for var in L:
            tmp.append(var[2])
        log_max[cpt]=max(tmp)
        #print("Le log max de la sequence {} est {}".format(cpt,max(tmp)))
        cpt+=1
        tmp=[]
        
    sorted_log_max=sorted(log_max.items(),key= lambda x:x[1],reverse=True)
    print("taille:",len(sorted_log_max))
    #log_max=sorted(log_max,key=None,reverse=True)
    return sorted_log_max
        
res=get_max_log(sequences,PWM,f_0,k)
print(res)

len :  78
taille: 78
[(0, 3.5441797599110556), (4, 3.5441797599110556), (15, 3.5441797599110556), (23, 3.5441797599110556), (24, 3.5441797599110556), (30, 3.5441797599110556), (31, 3.5441797599110556), (36, 3.5441797599110556), (38, 3.5441797599110556), (41, 3.5441797599110556), (42, 3.5441797599110556), (47, 3.5441797599110556), (49, 3.5441797599110556), (55, 3.5441797599110556), (57, 3.5441797599110556), (62, 3.5441797599110556), (63, 3.5441797599110556), (71, 3.5441797599110556), (72, 3.5441797599110556), (76, 3.5441797599110556), (1, 3.3066185709622427), (5, 3.3066185709622427), (8, 3.3066185709622427), (2, 3.0216468974766957), (6, 3.0216468974766957), (13, 3.0216468974766957), (20, 3.0216468974766957), (25, 3.0216468974766957), (27, 3.0216468974766957), (33, 3.0216468974766957), (40, 3.0216468974766957), (56, 3.0216468974766957), (60, 3.0216468974766957), (3, 2.9087875458341164), (10, 2.9087875458341164), (22, 2.9087875458341164), (34, 2.9087875458341164), (39, 2.9087875458341164)

In [60]:
cpt = 0
#print(positions_motifs["Sequence n° 1"])
for sequence, L in positions_motif.items():
    #print ( sequence, ", nombre de motifs trouvés :", len ( L ) )
    cpt += len (  L)
print ( "Nombre de motifs trouvés:", cpt )


Nombre de motifs trouvés: 2828


### D -  Localisation de facteur de transcriptions
15\. Une fois que vous avez localisé le motif dans le donnée de chipSeq, nous allons identifier les fonctions de gènes qui sont associées aux facteurs de transcription. Pour cela, nous allons utiliser le fichier qui contient tout le génome (C_glabrata_1000bp_only.fasta), localiser le motif (vous pouvez utiliser un des algos précédent), et identifier les gènes. Ensuite nous pouvons vérifier leurs fonctions dans le fichier "C_glabrata_annotations.csv" à l'aide de ses identifiant.

In [114]:
sequences_glabrata = readFasta ( "C_glabrata_1000bp_only.fasta" )

In [115]:
afficher_nb_motifs_trouvés ( sequences_glabrata, search )


Le motif CCAAT a été trouvé dans 3735 séquences différentes.


In [125]:
k=5
#print(p)
res=get_max_log(sequences_glabrata,PWM,f_0,k)
print(res[0:10])
#On récupère les 10 premières séquences avec le plus grand log

len :  5299
taille: 5299
[(0, 3.5441797599110556), (3, 3.5441797599110556), (5, 3.5441797599110556), (6, 3.5441797599110556), (7, 3.5441797599110556), (9, 3.5441797599110556), (10, 3.5441797599110556), (12, 3.5441797599110556), (13, 3.5441797599110556), (14, 3.5441797599110556)]


Les séquences avec le log le plus élevé et leur locus tag:

-Séquence 0 : CAGL0I00616g : 	Ortholog(s) have protein transmembrane transporter activity, role in posttranslational protein targeting to membrane, translocation and Sec62/Sec63 complex localization

-Séquence 3 : CAGL0L11704g : DNA Binding, DNA secondary structure binding activity

-Séquence 5:  CAGL0L07810g : have role in cellular potassium ion homeostasis 

-Séquence 6:  CAGL0L10230g :  chaperone binding, protein-macromolecule adaptor activity, role in cAMP-mediated signaling, cellular protein-containing complex assembly, kinetochore assembly and cytoplasm, nucleus, ubiquitin ligase complex localization

-Séquence 7:  CAGL0K09856g : have S-adenosylmethionine-dependent methyltransferase activity and role in protein methylation

-Séquence 9:  CAGL0M01144g : have nuclear export signal receptor activity, role in protein export from nucleus, tRNA re-export from nucleus and cytoplasm, nucleus localization

-Séquence 10: CAGL0M09735g : Putative protein involved in DNA damage checkpoint; gene is upregulated in azole-resistant strain

-Séquence 12: CAGL0E03894g :  Glycerol transporter; involved in flucytosine resistance; double fps1/fps2 mutant accumulates glycerol, has constitutive cell wall stress, is hypersensitive to caspofungin in vitro and in vivo

-Séquence 13: CAGL0I02222g :DNA helicase activity, chromatin binding and nuclease activity

-Séquence 14: CAGL0C01441g : role in endocytic recycling, tRNA methylation, wobble position ribose methylation and cytoplasm, endosome localization



